In [5]:
import tensorflow as tf
from tensorflow.keras.layers import (
    Input,
    IntegerLookup,
    Embedding,
    Flatten,
    Dot,
)
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

import pandas as pd

In [6]:
TRAIN_PATH = "../local/train.csv"
TEST_PATH = "../local/test.csv"
MODEL_PATH = "../local/model.h5"

In [7]:

def load_model():
    """
    A dummy pre-trained model for testing purposes.
    """
    # Assuming you have a CSV file with columns: 'user_id', 'movie_id', 'rating'
    train = pd.read_csv(TRAIN_PATH)
    test = pd.read_csv(TEST_PATH)

    # Unique users and movies
    all_users = train["user_id"].unique()
    all_movies = train["movie_id"].unique()

    # Create user and movie input layers
    user_input = Input(shape=(1,), name="user_input")
    movie_input = Input(shape=(1,), name="movie_input")

    # Create user and movie IntegerLookup
    user_as_integer = IntegerLookup(vocabulary=all_users)(user_input)
    movie_as_integer = IntegerLookup(vocabulary=all_movies)(movie_input)

    # Create user and movie embeddings
    user_embedding = Embedding(input_dim=len(all_users) + 1, output_dim=32)(user_as_integer)
    movie_embedding = Embedding(input_dim=len(all_movies) + 1, output_dim=32)(
        movie_as_integer
    )

    # Create the recommendation model (dot product of user and movie embeddings)
    dot_product = Dot(axes=2)([user_embedding, movie_embedding])

    # Flatten the dot_product
    flatten = Flatten()(dot_product)

    # Build and compile the model
    model = Model(inputs=[user_input, movie_input], outputs=flatten)
    model.compile(loss="mean_squared_error", optimizer="adam")

    return model

In [8]:
# A dummy pre-trained model
model = load_model()

# Load a small batch of data
train = pd.read_csv(TRAIN_PATH, nrows=10000) 
test = pd.read_csv(TEST_PATH, nrows=10000)

# Train the model
early_stop = EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True)
checkpoint = ModelCheckpoint(MODEL_PATH, monitor="val_loss", save_best_only=True)
model.fit(
    [train["user_id"], train["movie_id"]],
    train["user_rating"],
    epochs=10,
    batch_size=64,
    validation_split=0.2,
    callbacks=[early_stop, checkpoint],
)

Epoch 1/10
125/125 [==============================] - 1s 3ms/step - loss: 13.7102 - val_loss: 14.3169
Epoch 2/10
 77/125 [=================>............] - ETA: 0s - loss: 13.6731

/com.docker.devenvironments.code/.venv/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


125/125 [==============================] - 0s 3ms/step - loss: 13.5462 - val_loss: 14.2202
Epoch 3/10
125/125 [==============================] - 0s 2ms/step - loss: 12.3926 - val_loss: 13.6838
Epoch 4/10
125/125 [==============================] - 0s 2ms/step - loss: 9.3356 - val_loss: 12.6020
Epoch 5/10
125/125 [==============================] - 0s 2ms/step - loss: 5.6904 - val_loss: 11.4976
Epoch 6/10
125/125 [==============================] - 0s 2ms/step - loss: 3.2669 - val_loss: 10.7477
Epoch 7/10
125/125 [==============================] - 0s 2ms/step - loss: 2.0940 - val_loss: 10.3185
Epoch 8/10
125/125 [==============================] - 0s 2ms/step - loss: 1.5140 - val_loss: 10.0543
Epoch 9/10
125/125 [==============================] - 0s 2ms/step - loss: 1.1879 - val_loss: 9.8974
Epoch 10/10
125/125 [==============================] - 0s 2ms/step - loss: 0.9846 - val_loss: 9.7886
